In [1]:
import pandas as pd
import numpy as np

In [2]:
ls data/sports_scores/

astros_1A10.csv*   Rice_10H80.csv*    Tex_15E40.csv*
dynamo_10H30.csv*  rockets_1A10.csv*  UH_10H50.csv*


In [3]:
ls data/crime_beat/

crime_10H30_10_17.csv*  crime_10H80_10_17.csv*  crime_1A10_10_17.csv*
crime_10H50_10_17.csv*  crime_15E40_10_17.csv*


In [4]:
games = 'data/sports_scores/dynamo_10H30.csv'

crime = 'data/crime_beat/crime_10H30_10_17.csv'

# scores df

In [5]:
df = pd.read_csv(games,index_col='full_date').sort_index(ascending=True)


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 137 entries, 2010-04-01 to 2017-11-21
Data columns (total 7 columns):
Unnamed: 0    137 non-null int64
home_team     137 non-null object
home_score    137 non-null int64
away_team     137 non-null object
away_score    137 non-null int64
season        137 non-null object
beat          137 non-null object
dtypes: int64(3), object(4)
memory usage: 8.6+ KB


In [7]:
## win column
df['win'] =  np.where(df['home_score'] > df['away_score'],1,0)

# only oposing name and win boolean
df = df[['away_team','win']]

# show

df.head()

,away_team,win
full_date,,
2010-04-01,Real Salt Lake,1
2010-04-10,LA Galaxy,0
2010-04-17,Chivas USA,1
2010-05-01,Sporting KC,1
2010-05-05,FC Dallas,0


# Crime df

In [8]:
crime_df = pd.read_csv(crime,index_col='Date_Hour').sort_index(ascending=True)

# selectd columns
crime_df = crime_df[['OffenseType']]

# count Offense type given date
date_crimes = crime_df.groupby(crime_df.index)['OffenseType',].count()

## show
date_crimes.head()

,OffenseType
Date_Hour,
2010-01-01 00:00:00,1
2010-01-01 19:00:00,1
2010-01-01 20:00:00,1
2010-01-02 01:00:00,1
2010-01-03 04:00:00,1


In [9]:
#crime_df = pd.get_dummies(crime_df,drop_first=True)

In [10]:
#crime_df.info()

# Days Df
- create a df with everyday timestamp from Jan 2010 to Dec 2017

In [11]:
days = pd.date_range(start='01/01/2010', end='12/31/2017')
days = pd.DataFrame(days)

days.columns = ['days']
days = days.set_index('days').sort_index(ascending=True)
days.head()

""
days
2010-01-01
2010-01-02
2010-01-03
2010-01-04
2010-01-05


In [12]:
365 * 8

2920

In [13]:
len(days)/365

8.005479452054795

In [14]:
days.tail()

""
days
2017-12-27
2017-12-28
2017-12-29
2017-12-30
2017-12-31


# Merge days df with crimes_df

In [15]:
calendar_crimes = pd.merge(days,date_crimes,  left_index=True, right_index=True, how='left')
calendar_crimes.head()

,OffenseType
days,
2010-01-01,1.0
2010-01-02,NaN
2010-01-03,NaN
2010-01-04,NaN
2010-01-05,NaN


In [16]:
calendar_crimes.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2922 entries, 2010-01-01 to 2017-12-31
Data columns (total 1 columns):
OffenseType    294 non-null float64
dtypes: float64(1)
memory usage: 125.7 KB


In [17]:
calendar_crimes.OffenseType.value_counts(dropna=False)

NaN     2628
 1.0     260
 2.0      26
 3.0       8
Name: OffenseType, dtype: int64

## Change nans with 0

In [18]:
calendar_crimes.OffenseType.replace( np.nan,0, inplace=True)

In [19]:
calendar_crimes.OffenseType.value_counts(dropna=False)

0.0    2628
1.0     260
2.0      26
3.0       8
Name: OffenseType, dtype: int64

# merge calendar_crimes with score

In [20]:
merge_data = pd.merge(calendar_crimes,df,  left_index=True, right_index=True, how='left')

# change column names
#merge_data.columns = ['offenses','away_team','win']

merge_data.head()

,OffenseType,away_team,win
days,,,
2010-01-01,1.0,NaN,NaN
2010-01-02,0.0,NaN,NaN
2010-01-03,0.0,NaN,NaN
2010-01-04,0.0,NaN,NaN
2010-01-05,0.0,NaN,NaN


In [21]:
merge_data.away_team.value_counts(dropna=False)

NaN                        2785
Sporting KC                  10
Columbus                      8
FC Dallas                     8
San Jose                      8
Chicago                       8
D.C. United                   8
LA Galaxy                     8
NY Red Bulls                  8
New England                   7
Montreal                      7
Colorado                      7
Philadelphia                  7
Toronto FC                    7
Real Salt Lake                7
Portland                      6
Vancouver                     5
Chivas USA                    3
Seattle                       2
Minnesota                     2
Orlando                       2
Sporting Kansas City          2
NYCFC                         1
New York Red Bulls            1
Philadelphia Union            1
Kansas City                   1
Montreal Impact               1
 Washington D.C. United       1
Portland Tumbers              1
Name: away_team, dtype: int64

## Cleanup colums

In [22]:
merge_data.columns  = ['crime', 'away_team', 'won']

In [23]:
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2922 entries, 2010-01-01 to 2017-12-31
Data columns (total 3 columns):
crime        2922 non-null float64
away_team    137 non-null object
won          137 non-null float64
dtypes: float64(2), object(1)
memory usage: 171.3+ KB


In [24]:
merge_data.crime.value_counts(dropna=False)

0.0    2628
1.0     260
2.0      26
3.0       8
Name: crime, dtype: int64

In [25]:
merge_data.crime = merge_data.crime.astype(int)

In [26]:
merge_data.crime.value_counts(dropna=False)

0    2628
1     260
2      26
3       8
Name: crime, dtype: int64

In [27]:
merge_data

,crime,away_team,won
days,,,
2010-01-01,1,NaN,NaN
2010-01-02,0,NaN,NaN
2010-01-03,0,NaN,NaN
2010-01-04,0,NaN,NaN
2010-01-05,0,NaN,NaN
2010-01-06,1,NaN,NaN
2010-01-07,0,NaN,NaN
2010-01-08,0,NaN,NaN
2010-01-09,1,NaN,NaN


# Display non-nans

In [28]:
merge_data.dropna(subset=['away_team', 'won'])

,crime,away_team,won
days,,,
2010-04-01,0,Real Salt Lake,1.0
2010-04-10,0,LA Galaxy,0.0
2010-04-17,0,Chivas USA,1.0
2010-05-01,1,Sporting KC,1.0
2010-05-05,0,FC Dallas,0.0
2010-05-22,1,D.C. United,1.0
2010-05-29,0,Philadelphia,0.0
2010-06-26,1,Colorado,0.0
2010-07-10,0,Columbus,0.0


In [29]:
merge_data.to_csv('data/final/Dynamo1017_final.csv')

In [30]:
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2922 entries, 2010-01-01 to 2017-12-31
Data columns (total 3 columns):
crime        2922 non-null int64
away_team    137 non-null object
won          137 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 171.3+ KB
